# PVLIB usage example

LOCALISATION GEOGRAPHIQUE
longitude = 2.8856
latitude = 42.7983

## Initialization and imports

In [20]:
import pandas as pd
import pvlib
from pvlib import pvsystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain

## Defining constants

In [21]:
timezone = 'Europe/Paris'

In [22]:
locriv = Location(2.8856, 42.7983, 'Europe/Paris', 50, 'Rivesaltes')

In [23]:
temp_air = 20
wind_speed = 0

In [24]:
locriv

Location: 
  name: Rivesaltes
  latitude: 2.8856
  longitude: 42.7983
  altitude: 50
  tz: Europe/Paris

## Loading PV module

In [25]:
sandia_modules = pvsystem.retrieve_sam('SandiaMod')
cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

In [26]:
sandia_module = sandia_modules['Sharp_ND_216U2__2007__E__']
cec_inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_']

## Defining a PVSystem

In [27]:
system = pvsystem.LocalizedPVSystem(surface_tilt=20,
                                    surface_azimuth=180,
                                    latitude=locriv.latitude,
                                    longitude=locriv.longitude,
                                    name="Rivesaltes",
                                    altitude=50,
                                    tz=locriv.tz,
                                    module_parameters=sandia_module,
                                    inverter_parameters=cec_inverter)

In [28]:
mc = ModelChain(system, locriv)

In [29]:
print(mc)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses


Next, we run a model with some simple weather data.

In [30]:
weather = pd.DataFrame([[1050, 1000, 100, 30, 5]], 
                       columns=['ghi', 'dni', 'dhi', 'temp_air', 'wind_speed'], 
                       index=[pd.Timestamp('20170401 1200', tz='US/Arizona')])

mc.run_model(times=weather.index, weather=weather)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses

ModelChain stores the modeling results on a series of attributes. A few examples are shown below.

In [31]:
mc.aoi, mc.dc, mc.ac

(2017-04-01 12:00:00-07:00    145.31646
 Name: aoi, dtype: float64,
                            i_sc  i_mp  v_oc  v_mp  p_mp  i_x  i_xx
 2017-04-01 12:00:00-07:00   0.0   0.0   0.0   NaN   NaN  0.0   0.0,
 2017-04-01 12:00:00-07:00   NaN
 dtype: float64)

## PV module simulation example

In [69]:
naive_times = pd.DatetimeIndex(start='2015', end='2016', freq='1h', timezone=timezone)

In [73]:
clearsky = system.get_clearsky(naive_times)

In [75]:
solar_position = system.get_solarposition(naive_times)

In [79]:
total_irrad = system.get_irradiance(solar_position['apparent_zenith'],
                                                      solar_position['azimuth'],
                                                      clearsky['dni'],
                                                      clearsky['ghi'],
                                                      clearsky['dhi'])

In [82]:
temps = system.sapm_celltemp(total_irrad['poa_global'],
                                               wind_speed, temp_air)

In [84]:
aoi = system.get_aoi(solar_position['apparent_zenith'],
                               solar_position['azimuth'])

In [85]:
airmass = system.get_airmass(solar_position=solar_position)

In [86]:
effective_irradiance = system.sapm_effective_irradiance(
    total_irrad['poa_direct'], total_irrad['poa_diffuse'],
    airmass['airmass_absolute'], aoi)

In [88]:
dc = system.sapm(effective_irradiance, temps['temp_cell'])
ac = system.snlinverter(dc['v_mp'], dc['p_mp'])
annual_energy = ac.sum()

In [89]:
annual_energy

428046.96266820782